# Organizando as imagens com nomes organizados

In [11]:
import os

# Caminhos para as pastas X (imagens) e Y (máscaras)
x_path = '/content/drive/MyDrive/OrganizedImages/Matched/X'
y_path = '/content/drive/MyDrive/OrganizedImages/Matched/Y'

# Listar arquivos em ambas as pastas
x_files = sorted(os.listdir(x_path))
y_files = sorted(os.listdir(y_path))

# Função para padronizar os nomes dos arquivos
def padronizar_nomes(x_files, y_files):
    for i, (x_file, y_file) in enumerate(zip(x_files, y_files)):
        # Gerar novo nome padrão
        novo_nome = f"imagem_{i+1:04d}"  # Exemplo: "imagem_0001"

        # Obter a extensão original dos arquivos
        x_ext = os.path.splitext(x_file)[1]
        y_ext = os.path.splitext(y_file)[1]

        # Novos nomes completos
        novo_x_nome = f"{novo_nome}_x{x_ext}"
        novo_y_nome = f"{novo_nome}_y{y_ext}"

        # Caminhos completos para os arquivos antigos e novos
        x_antigo_caminho = os.path.join(x_path, x_file)
        y_antigo_caminho = os.path.join(y_path, y_file)

        x_novo_caminho = os.path.join(x_path, novo_x_nome)
        y_novo_caminho = os.path.join(y_path, novo_y_nome)

        # Renomear os arquivos
        os.rename(x_antigo_caminho, x_novo_caminho)
        os.rename(y_antigo_caminho, y_novo_caminho)

        print(f"Renomeado: {x_file} -> {novo_x_nome}")
        print(f"Renomeado: {y_file} -> {novo_y_nome}")

# Chamar a função para padronizar os nomes
padronizar_nomes(x_files, y_files)


A saída de streaming foi truncada nas últimas 5000 linhas.
Renomeado: 02-08-2024_video2_00000000210000100.mp4_0071_1_x.png -> imagem_0338_x.png
Renomeado: 02-08-2024_video2_00000000210000100.mp4_0071_1_y.png -> imagem_0338_y.png
Renomeado: 02-08-2024_video2_00000000210000100.mp4_0072_1_x.png -> imagem_0339_x.png
Renomeado: 02-08-2024_video2_00000000210000100.mp4_0072_1_y.png -> imagem_0339_y.png
Renomeado: 02-08-2024_video2_00000000210000100.mp4_0078_1_x.png -> imagem_0340_x.png
Renomeado: 02-08-2024_video2_00000000210000100.mp4_0078_1_y.png -> imagem_0340_y.png
Renomeado: 02-08-2024_video2_00000000210000100.mp4_0092_1_x.png -> imagem_0341_x.png
Renomeado: 02-08-2024_video2_00000000210000100.mp4_0092_1_y.png -> imagem_0341_y.png
Renomeado: 02-08-2024_video2_00000000210000100.mp4_0094_1_x.png -> imagem_0342_x.png
Renomeado: 02-08-2024_video2_00000000210000100.mp4_0094_1_y.png -> imagem_0342_y.png
Renomeado: 02-08-2024_video2_00000000210000100.mp4_0106_1_x.png -> imagem_0343_x.png
Renome

In [12]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import os
import cv2
import numpy as np

# Definimos os caminhos para onde estão nossas imagens (X) e máscaras (Y).
# O output_labels_path é a pasta onde vamos salvar os arquivos .txt com as anotações
x_path = '/content/drive/MyDrive/OrganizedImages/Matched/X'
y_path = '/content/drive/MyDrive/OrganizedImages/Matched/Y'
output_labels_path = '/content/drive/MyDrive/OrganizedImages/Matched/labels'

# Função que transforma a máscara em uma bounding box.
# Resumindo, a ideia é encontrar as áreas "brancas" na máscara, que correspondem às partes que queremos detectar (a cabeça ou os olhos do boi, por exemplo).
def mask_to_bounding_box(mask):
    """
    Converte uma máscara de segmentação em uma bounding box.
    """
    # Primeiro, aplicamos um "threshold" para deixar tudo ou branco ou preto.
    # Isso ajuda a separar bem as áreas de interesse do fundo.
    _, thresholded_mask = cv2.threshold(mask, 200, 255, cv2.THRESH_BINARY)

    # Agora, encontramos os contornos. A ideia aqui é identificar as bordas das áreas brancas que acabamos de isolar.
    contours, _ = cv2.findContours(thresholded_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Para cada contorno, calculamos uma "caixa delimitadora" (bounding box), que nada mais é que um retângulo que envolve toda a área branca.
    bounding_boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append((x, y, w, h))  # Salvamos as coordenadas (x, y) e as dimensões (largura e altura)

    return bounding_boxes

# Função que salva as bounding boxes no formato YOLO.
# Esse formato requer que a gente normalize os valores (ou seja, transformar as coordenadas da bounding box em uma fração do tamanho da imagem).
def save_yolo_annotations(bounding_boxes, image_width, image_height, save_path):
    annotations = []  # Aqui vamos armazenar todas as anotações para uma imagem.
    for (x, y, w, h) in bounding_boxes:
        # Calculamos o centro da bounding box e normalizamos pelo tamanho da imagem.
        center_x = (x + w / 2) / image_width
        center_y = (y + h / 2) / image_height
        width = w / image_width
        height = h / image_height

        # Cada linha do arquivo .txt vai ter o seguinte formato:
        # classe_id (sempre 0, porque só temos uma classe: "cabeça"), centro_x, centro_y, largura, altura.
        annotations.append(f"0 {center_x} {center_y} {width} {height}\n")

    # Agora, salvamos essas anotações em um arquivo .txt
    with open(save_path, 'w') as f:
        f.writelines(annotations)

# Garantimos que o diretório para salvar os arquivos de labels exista.
# Se ele não existir, a função os.makedirs vai criá-lo para a gente.
os.makedirs(output_labels_path, exist_ok=True)

# Agora, iteramos sobre todos os arquivos de máscara e geramos as bounding boxes para cada uma delas.
y_files = sorted(os.listdir(y_path))
x_files = sorted(os.listdir(x_path))

for i, y_file in enumerate(y_files):
    # Carregamos a máscara. Se der algum erro, pulamos esse arquivo.
    mask = cv2.imread(os.path.join(y_path, y_file), cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"Erro ao carregar a máscara: {y_file}")
        continue

    # Aqui pegamos as bounding boxes detectadas na máscara.
    bounding_boxes = mask_to_bounding_box(mask)

    # Pegamos as dimensões da imagem. Supondo que imagem e máscara têm o mesmo tamanho.
    image_width, image_height = mask.shape[1], mask.shape[0]

    # Geramos o nome do arquivo .txt onde vamos salvar as anotações da imagem correspondente.
    label_filename = os.path.join(output_labels_path, f"imagem_{i+1:04d}.txt")

    # Se a função encontrou bounding boxes, salvamos as anotações no arquivo .txt.
    if bounding_boxes:
        save_yolo_annotations(bounding_boxes, image_width, image_height, label_filename)
        print(f"Anotações salvas em: {label_filename}")
    else:
        print(f"Nenhuma bounding box encontrada para {y_file}")


Anotações salvas em: /content/drive/MyDrive/OrganizedImages/Matched/labels/imagem_0001.txt
Anotações salvas em: /content/drive/MyDrive/OrganizedImages/Matched/labels/imagem_0002.txt
Anotações salvas em: /content/drive/MyDrive/OrganizedImages/Matched/labels/imagem_0003.txt
Anotações salvas em: /content/drive/MyDrive/OrganizedImages/Matched/labels/imagem_0004.txt
Anotações salvas em: /content/drive/MyDrive/OrganizedImages/Matched/labels/imagem_0005.txt
Anotações salvas em: /content/drive/MyDrive/OrganizedImages/Matched/labels/imagem_0006.txt
Anotações salvas em: /content/drive/MyDrive/OrganizedImages/Matched/labels/imagem_0007.txt
Anotações salvas em: /content/drive/MyDrive/OrganizedImages/Matched/labels/imagem_0008.txt
Anotações salvas em: /content/drive/MyDrive/OrganizedImages/Matched/labels/imagem_0009.txt
Anotações salvas em: /content/drive/MyDrive/OrganizedImages/Matched/labels/imagem_0010.txt
Anotações salvas em: /content/drive/MyDrive/OrganizedImages/Matched/labels/imagem_0011.txt